# Step to test  

- Check tasks.py for basic setup
- Run the below command
```bash
# docker-compose terminal
docker-compose -f docker-compose.yml up
```

```bash
# celery terminal
celery -A celery_workers.controller.celery_app worker -l info
```

In [1]:
from celery_workers.tasks import start_pipeline

In [4]:
urls = [
    "https://docs.celeryq.dev/en/stable/getting-started/introduction.html",
    "https://docs.celeryq.dev/en/stable/getting-started/first-steps-with-celery.html",
    "https://lip17.medium.com/hands-on-learn-python-celery-in-30-minutes-9544aabb70b1",
    "https://dev.to/tecnosam/celery-pythons-silent-hero-4839",
    "https://en.wikipedia.org/wiki/Celery_(software)",
    "https://docs.celeryq.dev/en/stable/getting-started/introduction.html",
]

start_pipeline(urls).get()

'Delivered'

In [2]:
from celery_workers.agents_demo import start_agent

In [3]:
initial_context = {"history": [], "query": "How to test Celery agents?"}
start_agent.apply_async(args=[initial_context])
print("Autonomous agent chain launched")

Autonomous agent chain launched


In [2]:
from package.database.web_register.db import WebDB

In [3]:
webDB = WebDB(db_name="research-rocket.db")

In [4]:
urls = webDB.read_all()['url'].tolist()
urls[0:10]

['https://www.quora.com/What-did-ancient-Egyptian-pharaohs-do-each-day',
 'https://www.ancient-origins.net/history-famous-people/egyptian-pharaoh-life-0019086',
 'https://www.worldhistory.org/article/933/daily-life-in-ancient-egypt/',
 'https://www.historyskills.com/classroom/ancient-history/anc-role-of-pharaoh-reading/?srsltid=AfmBOopJLuPJDBdTlyz6Q1i2UKrAyYZeYKph8gKWfipaWN7OeJvt-npQ',
 'https://www.quora.com/What-was-it-like-to-live-as-a-pharaoh-in-ancient-Egypt-I-mean-like-daily-situations-and-such-What-obligations-did-they-have-and-how-did-they-communicate-with-their-subjects',
 'https://en.wikipedia.org/wiki/Functions_of_the_Pharaoh',
 'https://vocal.media/fiction/insane-life-of-an-egyptian-pharaoh',
 'https://en.wikipedia.org/wiki/Pharaoh',
 'https://www.twinkl.com.sg/blog/ancient-egypt-10-facts-about-pharaohs',
 'https://en.wikipedia.org/wiki/Ninja']

In [6]:
%%time
session_id = "mock-id"
start_pipeline(session_id,urls[:50]).get()

CPU times: user 625 ms, sys: 188 ms, total: 814 ms
Wall time: 5.8 s


'Delivered'

In [7]:
from broai.experiments.huggingface_embedding import BAAIEmbedding
baai_em = BAAIEmbedding()

/home/ec2-user/SageMaker/proxy/broai-research-rocket/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_24665/3805726944.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 189216.72it/s]


In [8]:
from broai.experiments.vector_store import DuckVectorStore
knowledgeDB = DuckVectorStore(
            db_name="test.db",
            table="knowledge",
            embedding=baai_em
        )

/tmp/ipykernel_24665/3832052936.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  knowledgeDB = DuckVectorStore(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [9]:
from broai.experiments.web_scraping import scrape_by_jina_ai
from broai.experiments.cleanup_markdown import clean_up_markdown_link

In [14]:
import json
import os

def load_all_json_files(session_id: str):
    dir_path = os.path.join("tmp", session_id)
    data_list = []

    if not os.path.exists(dir_path):
        print(f"No folder found for session: {session_id}")
        return data_list

    for filename in os.listdir(dir_path):
        if filename.endswith(".json"):
            file_path = os.path.join(dir_path, filename)
            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                    data_list.append(data)
                except json.JSONDecodeError as e:
                    print(f"Skipping {filename}: Invalid JSON - {e}")

    return data_list

# Example usage
session_id = "mock-id"
all_data = load_all_json_files(session_id)

In [15]:
len(all_data)

50

In [19]:
from broai.interface import Context
from broai.experiments.chunk import split_overlap

In [24]:
list({"test":"tost"}.keys())[0]

'test'

In [25]:
all_contexts = []
for data in all_data:
    url = list(data.keys())[0]
    context = Context(context=data[url], metadata={"source": url})
    contexts = split_overlap([context])
    all_contexts.extend(contexts)

/tmp/ipykernel_24665/1597514014.py:5: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  contexts = split_overlap([context])


In [26]:
len(all_contexts)

206

In [27]:
%%time

knowledgeDB.add_contexts(all_contexts)

CPU times: user 8.68 s, sys: 106 ms, total: 8.78 s
Wall time: 8.49 s
